In [9]:
import sys
import yaml
import random
import requests
import functools
import os.path

try:
    data = yaml.load(open('tests.yml'))
except IOError as e:
    print('Failed to open test definition file "tests.yml":', str(e), file=sys.stderr)
    data = dict()
base_url = data.get('base_url', 'http://localhost:5000/')

def login(email):
    if email == 'PUT_YOUR_EMAIL_HERE':
        print("Please replace the text 'PUT_YOUR_EMAIL_HERE' with your email address")
        return None
    r = requests.post(base_url + 'getkey', json=dict(email=email))
    if r.status_code == 200:
        key = r.json().get('key')
    else:
        print(r.status_code)
        return None
    print("your email is:", email, "your key is:", key)
    return key

In [10]:
def test_answer(func_name, key, *parameters):
    correct = False

    if key == None:
        return correct
    if func_name in globals():
        try:
            func = globals()[func_name]
            answer = func(*parameters)
            print(func_name, 'answer:', answer, file=sys.stderr)
        except:
            pass
        else:
            r = requests.post(base_url + 'test_' + func_name, json=dict(key=key, parameters=parameters, answer=answer))               
            if r.status_code == 200:
                correct = r.json().get('correct', False)
                if not correct:
                    print('Wrong answer. Correct answer:', r.json().get('correct_answer', 'unknown'))
            else:
                print("Failed to run test:", r.json().get('reason'), file=sys.stderr)
    return(correct)

try:
    data = yaml.load(open('tests.yml'))
except IOError as e:
    print('Failed to open test definition file "tests.yml":', str(e), file=sys.stderr)
    data = dict()
else:
    for name in data.get('tests', dict()).keys():
        test_function_name = 'test_' + name
        globals()[test_function_name] = functools.partial(test_answer, name)

In [16]:
def get_marks(key):
    if key is not None:
        r = requests.get(base_url + 'getmark/' + key)
        if r.status_code == 200:
            total_marks = r.json().get('total', 0)
            correct_answers = r.json().get('tests_correct', [])
            print("total marks:", total_marks)
            if total_marks > 0:
                print("tests correct:", correct_answers)

In [12]:
%%javascript
// YOU HAVE TO RUN THIS CELL MANUALLY, IT ISN'T RUN WHEN YOU DO 'Cell -> Run All'
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_PATH = '" + nb.notebook_path + "'";
kernel.execute(command)

<IPython.core.display.Javascript object>

In [21]:
def submit_notebook(key):
    if key is not None:
        nb_content = open(NOTEBOOK_PATH).read()
        r = requests.post(base_url + 'submit_notebook', json=dict(key=key, notebook=nb_content))
        if r.status_code == 200:
            print("notebook submitted successfully")
        else:
            print("notebook submission failed")

### The test starts here

To start the test, replace the text `PUT_YOUR_EMAIL_HERE` in the cell below with your email address and run all the cells in the notebook with Cell -> Run All

In [19]:
# put your email here
email = 'PUT_YOUR_EMAIL_HERE'
key = login(email)

Please replace the text 'PUT_YOUR_EMAIL_HERE' with your email address


#### Test problems

**Problem 1.** Write a function named `product_of_cubes` that takes a single parameter, a number `n`. Calling `product_of_cubes(n)` should return the product of the cubes of the numbers from 1 to n. E.g. `product_of_cubes(3)` should calculate `(1 ** 3) * (2 ** 3) * (3 ** 3)` which is `1 * 8 * 27` which is `216`. **1 mark**

**Problem 2.** Write a function `add_decoration(string1, string2)` that takes two string parameters and returns the contents of `string1` with `string2` on either side. E.g. `add_decoration("AAA", "^")` should return "^AAA^". **1 mark**

**Problem 3.** The Python string method `.count()` counts the non-overlapping occurences of a substring in a string. So, for example, the following code:

    mystring = 'eat the easter egg'
    num = mystring.count('ea')
    print(num)

prints `2`. Write a function `motif_count(dnastring, motif)` that when given a DNA string (the first parameter) returns the count of how often a motif (the second parameter) is present in this string (not counting overlapping occurences). The DNA string could contain both uppercase and lowercase characters and the function should not let that confuse the count it returns. For example, `motif_count('gcttccatccaacatctcagcatgatgaactt', 'TT')` should return `2`. **1 mark**

**Problem 4.** Write a function `complement(dnastring)` that returns the complement of a DNA string. E.g. `complement('GATTACA')` would return `'CTAATGT'`. **1 mark**

**Problem 5.** A DNA string e.g. `'ATAGTAGAATTCTAT'` is cut using the restriction enzyme *EcoRI*. This enzyme has the motif G\*AATTC, where the \* is the cutting point. Write a function `piece_lengths(base_string)` which, when passed a DNA string base string returns the lengths of both pieces of DNA that the base string would be cut into. E.g. for the string `'ATAGTAGAATTCTAT'` it would return `[7, 8]`. If the enzyme motif is not found the function should return an empty list (`[]`). Assume the DNA strings are all a single case (uppercase or lowercase). **3 marks**

**Problem 6**. Write a function `remove_adapter(dnalist, adapter)` that is given two parameters, a list of DNA strings and an adapter. It should search for the adapter at the start of each DNA string and if it is found, remove it from the string. The function should return the list of strings with any adapters removed if they are present. For example, if given the list `["GATTACATAAAAAGCTTCCATCCAACATCTCAGCATGATGAAACTT", "TCTAGCCTTACAAGTACTAACCGGCTTTTTCCTAGCCGTCCATTAC"]` as the first parameter and the adapter `GATTACA` as the second parameter, `remove_adapter` should return `['TAAAAAGCTTCCATCCAACATCTCAGCATGATGAAACTT', 'TCTAGCCTTACAAGTACTAACCGGCTTTTTCCTAGCCGTCCATTAC']`. Assume the DNA strings are all the same case. **3 marks**

Start your own work in the cell below. You may use as many cells as you wish.

When you have completed your work, evaluate the cell below this one.

In [23]:
if test_complement(key, 'GATTACA'):
    print("complement test 1 works")
if test_complement(key, 'ATCCTCTTGTGGATTGGGGCCATACTYTGCTTCTTAGCCTACGGCATCCAGGCAGGGACC'):
    print("complement test 2 works")
if test_add_decoration(key, 'GREETINGS', '^^^'):
    print('decoration test 1 works')
if test_product_of_cubes(key, 10):
    print('product_of_cubes test 1 works')
if test_motif_count(key, 'gcttccatccaacatctcagcatgatgaactt', 'TT'):
    print('motif_count test 1 works')
if test_piece_lengths(key, 'ATAGTAGAATTCTAT'):
    print('piece_lengths test 1 works')
if test_piece_lengths(key, 'ATAGTAAATTCTAT'):
    print('piece_lengths test 2 works')
adapter = 'GATTACA'
bases  = ["GATTACATAAAAAGCTTCCATCCAACATCTCAGCATGATGAAACTT", "TCTAGCCTTACAAGTACTAACCGGCTTTTTCCTAGCCGTCCATTAC"]
if test_remove_adapter(key, bases, adapter):
    print('remove_adapter test 1 works')
submit_notebook(key)
get_marks(key)